# Проект: Определение похожих товаров с использованием FAISS и оценка качества рекомендаций

## Описание проекта:

### Цель проекта

Создать систему, способную идентифицировать и рекомендовать пять товаров, наиболее схожих с каждым элементом из проверочного датасета (validation.csv), используя для этого основную базу данных товаров (base.csv). Эффективность и точность предлагаемых рекомендаций необходимо оценить по метрике accuracy@5.

### Используемые технологии

* FAISS для быстрого поиска по схожести в больших наборах данных.
* Scikit-learn для обучения моделей и оценки качества рекомендаций.

### План работы

* Шаг 1: Подготовка данных и анализ
  * Загрузка и первичный анализ данных из base.csv, validation.csv и validation_answer.csv.
  * Преобразование идентификаторов Id и Target из строк в числовые значения для упрощения обработки.
  * Разделение признаков и целевых переменных, нормализация данных.
* Шаг 2: Работа с FAISS
  * Создание и конфигурация GPU-оптимизированного индекса в FAISS для базового набора данных.
  * Поиск топ-5 похожих товаров для каждого элемента в валидационном наборе.
* Шаг 3: Подготовка обучающего датасета
  * Формирование обучающего датасета на основе результатов поиска с использованием FAISS, включая создание признаков, отражающих схожесть между товарами.
  * Определение целевой переменной (Target) на основе правильных ответов из validation_answer.csv.
* Шаг 4: Обучение модей
  * Оценка качества модели с использованием метрики accuracy@5.
* Шаг 5: Анализ результатов
  * Детальный анализ результатов предсказания модели.
  * Сравнение эффективности рекомендаций FAISS и точности классификации модели логистической регрессии.

## Подготовка данных

### Загрузка библиотек, таблиц, инициализация функций и констант

In [1]:
%pip install faiss_cpu

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
# Базовые библиотеки для работы с данными
import pandas as pd
import numpy as np
import faiss

# Визуализация
import seaborn as sns
import matplotlib.pyplot as plt

# Модели и инструменты машинного обучения
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.metrics import make_scorer, accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression, LogisticRegression

# Утилиты
from IPython.display import display
import warnings
import time


# Игнорирование предупреждений
warnings.filterwarnings('ignore')

In [3]:
pd.set_option('display.min_rows', 15)
pd.set_option('display.max_rows', 20)
pd.options.display.float_format = '{:,.2f}'.format

In [4]:
valid = pd.read_csv('validation.csv')

In [5]:
valid_target= pd.read_csv('validation_answer.csv')

In [6]:
train = pd.read_csv('train.csv')

In [7]:
base = pd.read_csv('base.csv')

In [8]:
# функция информации по таблице
def dataframe_summary(df, string):
    # Вывод общей информации
    print("Общая информация по таблице:", string)
    df.info()

    print("\n Статистическое описание:")
    display(df.describe().transpose())

    print("\nСлучайные примеры:")
    display(df.sample(5))

    print("\nКоличество строк и столбцов:", df.shape)

    print("\nКоличество явных дубликатов:", df.duplicated().sum())
    print('')

In [9]:
def downcast_dataframe(df):
    for col in df.columns:
        if df[col].dtype == 'float64':  # Проверяем, является ли тип столбца float64
            df[col] = pd.to_numeric(df[col], downcast='float')
        elif df[col].dtype == 'int64':  # Проверяем, является ли тип столбца int64
            df[col] = pd.to_numeric(df[col], downcast='integer')
    return df

Все данные загружены, можно приступать к работе.

### Обзор данных

In [10]:
for data in [valid, valid_target, train, base]:
    dataframe_summary(data, 'Данные')

Общая информация по таблице: Данные
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 73 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Id      100000 non-null  object 
 1   0       100000 non-null  float64
 2   1       100000 non-null  float64
 3   2       100000 non-null  float64
 4   3       100000 non-null  float64
 5   4       100000 non-null  float64
 6   5       100000 non-null  float64
 7   6       100000 non-null  float64
 8   7       100000 non-null  float64
 9   8       100000 non-null  float64
 10  9       100000 non-null  float64
 11  10      100000 non-null  float64
 12  11      100000 non-null  float64
 13  12      100000 non-null  float64
 14  13      100000 non-null  float64
 15  14      100000 non-null  float64
 16  15      100000 non-null  float64
 17  16      100000 non-null  float64
 18  17      100000 non-null  float64
 19  18      100000 non-null  float64
 20  19      10000

,count,mean,std,min,25%,50%,75%,max
0,"100,000.00",-85.30,25.78,-190.35,-103.23,-85.30,-67.42,14.43
1,"100,000.00",7.67,4.96,-11.11,4.29,7.66,10.99,27.41
2,"100,000.00",-43.84,39.14,-217.54,-69.68,-43.23,-17.77,134.86
3,"100,000.00",-146.12,20.50,-220.05,-160.21,-146.08,-132.12,-57.38
4,"100,000.00",111.64,47.75,-81.20,79.10,111.96,143.90,302.07
5,"100,000.00",-73.27,28.52,-176.71,-92.79,-73.58,-53.86,50.26
6,"100,000.00",-441.40,279.24,-791.46,-740.62,-513.92,-202.57,109.63
...,...,...,...,...,...,...,...,...
65,"100,000.00",14.40,98.70,-157.59,-70.42,14.77,99.80,185.09
66,"100,000.00",67.79,1.84,60.38,66.55,67.80,69.04,75.40



Случайные примеры:


,Id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71
89980,189980-query,-105.72,8.66,-50.96,-146.31,131.84,-96.75,44.53,-138.24,125.22,64.77,-145.57,-65.36,-49.69,16.97,191.04,-4.84,-40.67,-9.66,-113.16,-79.37,-121.48,"1,362.29",-35.57,42.22,-16.29,813.77,-132.52,16.56,-104.76,137.34,-4.79,-164.83,47.79,-533.00,49.03,66.41,21.96,-145.82,-140.88,-17.83,12.24,12.44,-186.79,87.68,-34.68,66.47,-126.29,-180.83,18.12,3.78,-51.24,-26.30,61.54,-232.12,82.97,66.32,-132.21,7.92,-162.87,-290.05,-114.73,-122.53,-60.13,31.45,-145.39,53.74,70.17,47.23,128.77,102.67,-680.50,-10.56
11200,111200-query,-43.67,10.75,-49.78,-135.86,131.44,-96.60,-690.05,-59.18,121.89,91.55,-37.12,-31.38,-44.83,40.23,153.50,-33.95,2.36,-61.11,-209.59,-77.91,-85.80,"1,507.23",-20.62,109.27,67.79,743.67,-23.77,18.61,-46.79,160.20,-7.12,-96.22,57.24,-886.89,45.81,50.64,-4.94,-142.37,-110.11,-25.57,-5.04,4.09,-188.83,166.00,-21.35,11.24,-7.35,-120.25,101.77,-34.23,71.34,-59.12,64.38,19.27,93.01,82.69,-133.64,-42.10,-77.10,-337.44,-82.71,-122.52,-52.31,40.72,-117.15,-95.21,68.88,-66.98,22.57,97.24,-929.32,-61.13
4216,104216-query,-97.73,7.20,-104.96,-129.49,64.07,-55.41,-530.12,54.29,121.83,87.08,-114.46,-83.77,73.40,-4.26,148.34,-23.25,10.80,-41.66,-166.82,-78.62,-93.42,"1,178.36",5.62,101.68,23.60,813.77,-69.87,14.98,-86.94,137.47,10.74,-168.06,133.10,-709.45,-1.23,95.18,-42.32,-149.18,-115.98,-28.78,-10.56,-1.35,-115.66,92.56,-45.86,45.35,-60.07,-56.46,14.06,-18.28,-21.25,-23.36,67.31,-50.14,76.68,75.75,-122.91,-84.97,-124.08,-329.84,-41.60,-119.94,-114.12,28.08,-170.15,75.26,68.96,-60.27,-17.29,103.57,-413.29,-83.86
87902,187902-query,-116.85,10.21,27.86,-135.00,102.52,-107.08,-56.26,-79.16,127.12,119.32,-135.28,-98.14,-78.91,4.64,126.06,-16.56,7.37,57.43,-142.50,-78.29,-137.43,"1,342.64",15.32,18.97,41.93,813.77,-76.35,7.10,-116.77,158.16,-25.80,-135.91,154.94,-529.30,80.84,111.93,-30.02,-136.01,-118.44,-60.15,-17.29,7.22,-127.15,123.30,-31.37,115.24,-89.25,-138.83,19.30,15.08,9.38,-128.12,83.26,-70.66,73.12,70.91,-129.10,-75.04,-109.37,-145.17,44.97,-132.36,-27.59,30.24,-138.64,139.07,66.83,144.24,8.09,85.90,-345.00,-57.94
10204,110204-query,-134.82,3.51,3.32,-147.10,130.56,-44.50,-146.80,-132.76,120.74,126.36,-147.04,-155.31,-111.90,-70.15,138.97,-28.59,-14.62,-1.73,-110.64,-79.06,-122.35,"1,507.23",57.08,99.20,-4.54,813.77,-72.36,22.32,-117.90,140.60,-16.24,-185.41,71.32,-481.59,14.43,99.95,31.47,-149.30,-132.90,-49.39,-2.71,4.12,-81.46,104.38,-50.38,81.74,-136.97,-146.25,135.45,-45.49,153.15,68.04,71.41,-176.65,91.62,81.88,-126.00,46.01,1.65,-608.51,-142.88,-128.38,-75.93,53.32,-194.00,-104.57,69.60,48.25,38.30,130.41,"-1,074.46",-19.27



Количество строк и столбцов: (100000, 73)

Количество явных дубликатов: 0

Общая информация по таблице: Данные
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   Id        100000 non-null  object
 1   Expected  100000 non-null  object
dtypes: object(2)
memory usage: 1.5+ MB

 Статистическое описание:


...


Количество строк и столбцов: (100000, 74)

Количество явных дубликатов: 0

Общая информация по таблице: Данные
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2918139 entries, 0 to 2918138
Data columns (total 73 columns):
 #   Column  Dtype  
---  ------  -----  
 0   Id      object 
 1   0       float64
 2   1       float64
 3   2       float64
 4   3       float64
 5   4       float64
 6   5       float64
 7   6       float64
 8   7       float64
 9   8       float64
 10  9       float64
 11  10      float64
 12  11      float64
 13  12      float64
 14  13      float64
 15  14      float64
 16  15      float64
 17  16      float64
 18  17      float64
 19  18      float64
 20  19      float64
 21  20      float64
 22  21      float64
 23  22      float64
 24  23      float64
 25  24      float64
 26  25      float64
 27  26      float64
 28  27      float64
 29  28      float64
 30  29      float64
 31  30      float64
 32  31      float64
 33  32      float64
 34  33      float6

,count,mean,std,min,25%,50%,75%,max
0,"2,918,139.00",-86.23,24.89,-199.47,-103.07,-86.23,-69.26,21.52
1,"2,918,139.00",8.08,4.95,-13.91,4.71,8.04,11.47,29.94
2,"2,918,139.00",-44.58,38.63,-240.07,-69.56,-43.82,-19.63,160.94
3,"2,918,139.00",-146.63,19.84,-232.67,-159.91,-146.78,-133.33,-51.37
4,"2,918,139.00",111.32,46.35,-105.58,80.51,111.87,142.37,319.66
5,"2,918,139.00",-71.99,28.19,-211.01,-91.38,-71.92,-52.44,58.81
6,"2,918,139.00",-392.22,271.65,-791.47,-629.33,-422.20,-156.67,109.63
...,...,...,...,...,...,...,...,...
65,"2,918,139.00",14.15,98.95,-157.59,-71.30,13.83,99.67,185.10
66,"2,918,139.00",67.79,1.82,59.51,66.58,67.81,69.03,75.71



Случайные примеры:


,Id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71
50380,50950-base,-149.02,2.05,-48.13,-156.86,117.90,-49.10,-759.63,-16.27,113.90,194.04,-159.99,-165.92,16.67,24.13,82.11,-6.73,-0.91,-8.82,-108.57,-80.55,-58.22,395.70,-19.12,102.63,42.40,375.07,-128.33,19.84,-66.95,145.52,-20.27,-134.24,106.51,-971.40,-47.81,73.21,-16.82,-142.86,-103.56,-63.81,-20.43,10.79,-137.07,70.08,-38.70,30.57,-64.01,-137.69,-5.14,-64.13,33.87,-15.49,70.95,-12.66,86.50,70.05,-132.80,35.76,-223.70,-393.57,121.21,-144.12,-46.34,19.15,-165.53,183.77,67.05,15.91,28.14,148.88,"-1,166.76",-25.16
1132499,1403229-base,-88.77,9.33,-45.09,-159.70,143.65,-83.32,-759.63,21.04,125.69,198.05,-194.63,-104.72,12.85,-2.57,123.30,-33.00,-7.04,-50.81,-183.25,-79.09,-96.02,"1,507.23",-52.94,8.67,15.06,813.77,-200.93,18.78,-120.94,147.29,-2.45,-71.36,84.76,-667.60,-12.70,78.54,-38.20,-128.11,-125.25,-44.17,-13.80,11.93,-94.59,121.57,-61.44,84.18,-55.81,-133.49,31.89,-41.74,66.72,-111.47,48.29,-100.37,86.52,57.55,-134.77,33.71,-207.29,-159.30,-222.15,-145.94,-112.01,25.25,-178.11,-93.40,66.15,55.70,34.25,120.59,-135.92,-67.97
123291,126650-base,-111.51,16.00,-53.16,-176.95,52.65,-28.19,12.54,112.42,126.56,111.22,-161.99,-101.39,-70.63,36.95,112.98,-10.44,42.95,-28.98,-127.91,-80.14,-125.05,"1,507.23",-88.37,74.16,35.25,813.77,-60.92,17.12,-105.18,142.36,19.35,-140.66,59.30,-537.18,23.06,118.71,-81.10,-161.51,-127.06,-31.69,-20.24,5.35,-202.96,108.32,-18.25,120.64,-26.78,-115.31,74.41,19.58,32.25,-25.02,74.50,-77.27,90.99,69.44,-135.54,-92.69,-171.00,-234.78,-119.51,-139.35,-92.08,-50.68,-154.07,180.96,67.27,15.76,103.73,121.04,-554.62,-59.44
277171,293906-base,-95.27,8.49,3.24,-138.36,63.77,-96.59,-752.96,15.01,129.39,182.44,-63.85,-44.64,-31.86,-4.72,150.08,-63.99,6.12,-2.63,-183.25,-78.82,-176.66,441.57,-21.24,104.35,39.22,813.77,39.86,17.56,-102.46,147.70,-5.93,-186.01,112.24,-529.30,-13.60,63.65,-41.41,-130.29,-109.62,-30.37,-3.22,5.96,-32.97,139.02,-61.24,74.48,-82.60,-165.05,14.40,22.27,45.88,-69.80,88.96,-183.45,88.46,73.95,-138.98,-31.20,-204.37,-217.04,-57.13,-116.47,-84.80,4.29,-200.89,-49.72,66.16,123.08,69.39,115.73,38.64,-20.98
1259737,1594335-base,-104.06,2.93,-43.18,-124.91,113.94,-67.94,-709.16,-38.33,107.42,188.81,-197.17,-141.51,-2.12,10.74,146.74,-8.58,-20.55,36.65,-129.84,-80.32,-85.77,"1,248.13",-1.29,37.97,25.65,813.77,-110.81,17.35,-104.18,139.40,22.75,-141.74,132.31,-529.30,17.70,82.36,-26.09,-153.36,-94.14,-56.95,-26.21,-7.94,-79.96,157.21,-34.25,53.77,-37.64,-213.99,77.74,-116.92,-1.59,-84.41,89.98,-53.23,88.65,72.72,-123.27,96.56,-225.61,-590.52,-52.79,-124.52,-120.62,0.23,-203.07,14.92,63.97,9.16,105.52,149.18,"-1,074.46",-30.85



Количество строк и столбцов: (2918139, 73)

Количество явных дубликатов: 0



* В данных отсутсвуют пропуски и дубликаты. 
* Наблюдается сильный разброс значений, понадобится масштабирование.
* Стоит отменить, что в `valid_target` в колонке `Expected` 91502 уникальных значений из 100000 строк. Значит есть товары, которые указали как схожие несколько раз.

Уменьшим потребляемый объем памяти данных. Переведем столбец Id в формат индекса. И уменьшим регистр называний столбцов.

In [11]:
base = downcast_dataframe(base)
base.columns = base.columns.str.lower()
base.set_index('id', inplace=True)

In [12]:
train = downcast_dataframe(train)
train.columns = train.columns.str.lower()
train.set_index('id', inplace=True)

In [13]:
valid = downcast_dataframe(valid)
valid.columns = valid.columns.str.lower()
valid.set_index('id', inplace=True)

In [14]:
valid_target = downcast_dataframe(valid_target)
valid_target.columns = valid_target.columns.str.lower()
valid_target.set_index('id', inplace=True)

In [15]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 100000 entries, 0-query to 99999-query
Data columns (total 73 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   0       100000 non-null  float32
 1   1       100000 non-null  float32
 2   2       100000 non-null  float32
 3   3       100000 non-null  float32
 4   4       100000 non-null  float32
 5   5       100000 non-null  float32
 6   6       100000 non-null  float32
 7   7       100000 non-null  float32
 8   8       100000 non-null  float32
 9   9       100000 non-null  float32
 10  10      100000 non-null  float32
 11  11      100000 non-null  float32
 12  12      100000 non-null  float32
 13  13      100000 non-null  float32
 14  14      100000 non-null  float32
 15  15      100000 non-null  float32
 16  16      100000 non-null  float32
 17  17      100000 non-null  float32
 18  18      100000 non-null  float32
 19  19      100000 non-null  float32
 20  20      100000 non-null  float32
 21  21  

Потребляемый объем памяти упал почти в два раза.

## Поиск ближайших соседей, формирование новой валидационной и тренировочной выборки

#### Алгоритм Faiss

In [16]:
# выделение цел. признака из обучающей выборки
train_target = train['target']
train.drop('target', axis=1, inplace=True)

In [17]:
# словарь id и номеров векторов
base_index = {k: v for k, v in enumerate(base.index.to_list())}

In [18]:
train_index = {k: v for k, v in enumerate(train.index.to_list())}

In [19]:
# Масштабирование 
scaler = StandardScaler()
scaler.fit(base)
base = scaler.transform(base)
train = scaler.transform(train)
valid = scaler.transform(valid)

In [20]:
N_CLUSTERS = 50   # количество кластеров
N_NEIGHBORS = 50   # количество ближайших соседей для поиска

In [21]:
%%time 
# Создаем квантизатор для определения кластеров
d = base.shape[1]  # размерность векторов
quantizer = faiss.IndexFlatL2(d)  

# Создаем индекс для векторов с использованием квантизатора
index = faiss.IndexIVFFlat(quantizer, d, N_CLUSTERS, faiss.METRIC_L2)

# Обучаем индекс на первых 50 000 строках из base
assert not index.is_trained  # убеждаемся, что индекс еще не обучен
index.train(base[:50000])

# Добавляем весь массив base в индекс после обучения
index.add(base)

# Включаем поиск по всему индексу, если не хватает векторов для заполнения кластеров
index.nprobe = 60   

CPU times: user 13.6 s, sys: 622 ms, total: 14.3 s
Wall time: 2.28 s


#### Создание выборки train

In [22]:
# Используем обученный индекс для поиска ближайших соседей в train
D, I = index.search(train, N_NEIGHBORS)

# D - расстояния до ближайших соседей, I - индексы ближайших соседей в массиве base

In [23]:
print(f'train_idx :\n {I[:3]} \n')
print(f'Distance:\n {D[:3]} \n')

train_idx :
 [[ 598613  755584  336969 1934845   13374 1136231  480296  583287 2360257
   450667  988777 1653095 1818641  503716 1631947   89840  143978  986050
   629775 2346335  223859  352715  455829 1747988 1674977  443428 2295048
   728097 1113711 1746258  751217  854264  792610 1375244  199196 2212973
  2398299 1679537  679629  684958  669932  108138 1285249  725525 1546520
   458465 1299775 1307670 2331892  547719]
 [ 737853   15185  519311  902634  223456  464427  220644 2146806 2524968
   643208  134844  301385  340012  298805  216389  188242  971689  573064
  1459914  548516  924284 1010871  917176 1348395   10845 2214631 1312093
   603976  458085  780936  419818  839888  426013 2375011  332349 1096992
   242607 2207064 1946481  838522 1084963 1238219  124343 1182284 1839713
   943287 2865631 2454898   49621 1541125]
 [ 173120 1784963 2863071 1143288  955831  137853 1368891 1162532  774247
  1984148 1479188 1238526 2351134  641280 1532675 1774212  948608   12304
  2395210 190

In [24]:
def accuracy_n(targets, predict, base_index):

    if isinstance(predict, np.ndarray):
        predict = predict.tolist()

    correct_predictions = sum(int(targets[i] in [base_index[idx] for idx in pred]) for i, pred in enumerate(predict))

    accuracy = (correct_predictions / len(predict)) * 100
    return accuracy

In [25]:
accuracy = accuracy_n(train_target, I, base_index)

In [26]:
print(f'Accuracy 50-neighbors: {accuracy:.2f}%')

Accuracy 50-neighbors: 76.76%


Создадим новые таблицы с таргет признаками, которые помогут в дальнейшем анализе.

In [27]:
# подготовка датафрейма с перечнем ID новых и старых объектов
idx_df = pd.DataFrame(data=I, index=train_index)
idx_df = pd.melt(idx_df.T)
idx_df.columns = ['id_query', 'predicted']
idx_df['id_predicted'] = [base_index[number] for number in idx_df['predicted'].values]
idx_df.drop('predicted', axis=1, inplace=True)
idx_df['id_query'] = idx_df['id_query'].map(train_index)


In [28]:
display(idx_df)

,id_query,id_predicted
0,0-query,675816-base
1,0-query,877519-base
2,0-query,361564-base
3,0-query,2725256-base
4,0-query,13406-base
5,0-query,1408780-base
6,0-query,530165-base
...,...,...
4999993,99999-query,4593791-base
4999994,99999-query,2638659-base


In [29]:
base_index_df = pd.Series(data=base_index)
train_index_df = pd.Series(data=train_index)

In [30]:
base_transform_df  = pd.DataFrame(base,  index=base_index_df)
train_transform_df = pd.DataFrame(train, index=train_index_df)

In [31]:
base_transform_df.sample(5)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71
532281-base,-0.86,1.48,-1.91,-0.67,-1.47,0.50,-1.35,0.13,1.16,0.80,0.43,-0.09,-1.41,-1.01,-0.51,0.42,1.05,1.94,-0.38,2.26,-0.75,0.62,-0.66,0.08,-0.44,0.42,1.18,-0.97,0.37,1.96,0.43,-0.63,-1.77,-0.54,-0.52,0.09,0.37,-1.52,-1.02,0.13,-1.18,-0.97,0.48,0.59,-1.15,1.58,-0.05,0.74,-1.19,-0.05,-1.61,0.75,0.44,-0.27,2.62,0.38,1.31,0.54,1.08,-0.54,-1.61,0.39,0.19,1.07,1.03,-0.27,0.31,-0.33,0.25,0.59,-0.71,1.14
2470812-base,1.13,0.50,1.78,0.44,-2.31,1.33,0.45,-0.68,0.48,0.99,-1.78,-0.30,-1.54,-0.90,-0.99,-0.24,-0.92,1.68,-0.45,0.84,0.78,0.62,-0.09,-0.36,-0.92,0.42,-0.32,-0.20,0.19,-1.28,0.02,-0.61,-0.82,0.28,-0.34,-0.49,-0.60,-0.25,1.00,-1.76,-1.43,-2.39,-0.46,0.16,1.34,-1.42,0.10,0.01,-2.48,-1.43,-0.87,0.25,0.45,-0.21,0.36,1.34,-0.20,-0.86,-0.38,0.87,2.01,1.73,-0.56,2.30,1.03,-1.61,-0.89,-2.95,0.49,-0.67,-0.71,-1.43
284697-base,-0.99,0.39,-1.03,-0.79,-0.67,1.13,-0.72,-0.16,0.37,0.27,1.01,1.11,1.80,0.70,0.67,-1.29,0.20,1.47,-0.23,0.59,0.48,0.62,-0.65,0.52,-0.43,-2.57,-0.38,0.87,-0.64,1.39,-1.41,-1.16,0.03,0.27,-1.71,0.74,1.16,1.45,-0.96,-0.19,-1.31,-0.44,0.48,0.60,0.44,1.49,0.25,2.02,1.89,-0.71,-0.71,0.21,1.00,0.34,0.46,-0.17,2.46,0.08,-1.22,0.63,-1.60,-0.36,0.14,0.94,1.44,-0.53,1.48,0.43,-0.18,0.34,1.12,-0.24
2972932-base,0.22,0.42,-0.62,-1.66,0.73,0.51,-0.39,-0.47,-1.59,-1.14,0.11,0.69,1.87,-0.91,0.91,-0.88,-1.55,-0.52,-1.49,0.82,0.22,0.62,-0.56,-1.49,1.42,0.42,-0.97,0.88,-1.74,1.61,-0.21,1.13,0.23,0.81,0.07,-0.26,1.79,0.48,1.30,0.94,-1.43,0.47,-1.16,2.21,1.26,1.99,-0.67,-0.91,-1.90,-0.65,0.73,-0.70,0.24,-0.46,0.03,0.68,0.10,-0.08,-0.72,-1.18,0.34,-0.46,-1.27,1.15,-0.84,-0.55,-0.35,-0.07,0.17,-0.42,-0.71,-1.32
268421-base,1.17,-0.23,1.02,0.73,0.60,-0.91,-0.43,0.07,1.40,-0.33,0.07,-0.30,-0.08,0.06,-0.05,-0.01,-0.05,0.75,0.61,0.64,0.57,0.62,-0.75,1.51,0.37,0.42,0.33,1.27,-0.54,-0.63,1.12,-0.06,0.41,1.40,0.67,1.00,-0.42,-0.06,-0.47,0.10,-0.39,-1.30,-0.59,-0.46,0.29,1.11,-0.14,0.08,-1.35,0.47,-0.39,-0.16,1.22,-1.25,-0.73,-0.19,0.82,0.65,-0.72,0.54,0.88,0.84,1.82,0.80,-0.29,-0.98,0.20,0.06,-0.54,0.37,-0.71,-0.90


In [32]:
train_transform_df.sample(5)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71
72994-query,1.16,1.68,-0.28,-1.11,-0.06,0.40,1.57,0.55,0.78,0.04,1.70,0.01,-1.28,-0.93,1.05,-1.15,-0.30,0.51,0.79,1.07,1.20,0.62,-0.08,0.18,-1.34,0.42,0.75,1.73,-0.65,-0.92,-0.76,0.23,-0.20,0.81,1.60,0.22,0.41,0.04,2.30,1.96,-1.44,-0.45,-0.70,2.62,0.04,-1.11,0.35,0.58,0.08,0.42,-1.06,1.99,-0.32,-0.82,0.50,-1.10,0.26,1.57,-0.73,1.67,-1.37,-0.72,0.04,-1.19,1.29,-1.25,0.86,0.04,0.07,-0.54,-0.71,0.63
52956-query,0.54,-0.26,-1.26,-0.44,-1.01,-0.16,1.09,0.90,2.29,0.35,-0.40,0.88,-1.92,1.98,0.46,1.45,-1.24,-0.05,-1.81,0.27,0.72,0.62,1.29,-0.68,0.17,0.42,0.60,-2.00,-0.34,-1.63,0.10,-0.04,0.03,1.53,-1.51,-0.62,-0.42,-0.60,1.09,-2.11,0.69,-1.97,-2.17,-1.05,-0.21,0.74,-0.65,-2.00,-2.57,1.35,1.13,0.43,-0.45,-0.23,-1.40,0.25,-0.38,1.46,0.35,0.12,0.71,0.03,2.08,1.16,-1.47,-0.95,-0.03,-1.49,1.29,-0.55,0.43,0.38
32702-query,1.76,0.58,1.51,2.05,0.66,1.57,-1.35,-0.76,-0.28,1.21,-1.89,1.43,1.64,1.10,0.00,0.28,-0.90,0.23,-0.26,1.78,0.14,0.62,-0.33,0.37,-0.68,-0.65,-0.27,0.37,-0.03,0.00,0.85,-0.67,-0.08,0.94,-1.94,0.22,-2.06,0.44,-0.67,-0.58,0.21,0.42,0.57,0.72,-1.18,-0.41,-0.82,-0.10,0.02,-0.46,-1.98,0.21,-0.64,-0.86,-1.32,-0.22,0.86,1.02,-1.51,-1.37,-1.12,0.95,-0.26,-0.60,-0.26,0.14,1.23,0.72,2.02,1.11,-0.30,-1.97
44817-query,1.32,-1.14,-0.28,0.29,-0.69,-0.06,-0.51,-0.36,-0.10,0.13,0.42,-0.70,-0.24,-0.17,1.29,-0.25,-0.57,-0.67,-1.35,-0.37,-0.07,-1.56,0.89,0.26,-0.87,0.42,0.42,3.06,-0.14,-1.65,-0.45,-0.53,1.07,-1.81,0.56,0.02,1.65,-1.02,1.04,0.86,-1.77,-0.45,-1.03,1.07,1.26,-0.47,0.85,0.65,-0.94,0.28,-1.59,-0.57,0.04,-0.65,-1.45,-1.74,0.40,1.41,-1.01,-1.17,0.83,-0.16,0.38,-0.26,1.07,-0.69,-0.20,-1.36,1.25,-0.60,-0.25,-0.01
46137-query,-0.41,-0.97,-0.63,-1.59,-2.78,1.30,0.28,1.24,0.38,0.40,1.94,-0.72,-1.76,-0.12,0.70,0.12,1.66,0.11,-0.09,-1.44,0.70,0.62,-0.51,-0.69,-0.11,0.42,-0.94,0.69,-0.65,0.03,-0.31,-0.89,-0.73,1.53,0.32,0.52,-2.43,-0.78,-0.36,0.61,-0.07,0.42,-0.70,-0.23,-0.26,1.94,-0.26,0.19,-0.67,0.89,-0.39,-0.37,1.03,-0.54,0.18,-1.40,1.10,-0.94,-1.59,1.58,-0.47,1.70,-0.13,-0.01,-1.34,-0.10,0.77,0.59,0.08,0.82,1.75,-0.21


In [33]:
train_new = pd.merge(idx_df, train_transform_df, left_on='id_query', right_index=True, how='left')


In [34]:
train_new = pd.merge(train_new, base_transform_df, left_on='id_predicted', right_index=True, how='left')

In [35]:
# Добавляем колонку 'target' в train_new, используя индексы из train_target для сопоставления
train_new['target'] = train_new['id_query'].map(train_target)
train_new['target'] = (train_new['target'] == train_new['id_predicted']).astype(int)
# выделение признаков и таргета
target_train = train_new['target']
features_train = train_new.drop(['id_query', 'id_predicted', 'target'] , axis=1)

Вывод:
Была создана тренировочная выборка на основе поиска 50 ближайших соседей. С точностью в 76%.

#### Создание валидацонной выборки

In [36]:
# Используем обученный индекс для поиска ближайших соседей в train
D, I = index.search(valid, N_NEIGHBORS)

In [37]:
valid_index = {k: v for k, v in enumerate(valid_target.index.to_list())}

In [38]:
valid_target_series = valid_target['expected']

In [39]:
valid_predictions = pd.DataFrame(I)  # I - индексы ближайших соседей
valid_predictions = valid_predictions.stack().reset_index(level=1, drop=True).to_frame('id_predicted')
valid_predictions['id_query'] = valid_predictions.index.map(valid_index)
valid_predictions['id_predicted'] = valid_predictions['id_predicted'].map(base_index)  # Переводим индексы в ID
valid_predictions.set_index('id_query', inplace=True)

In [40]:
merged_valid = valid_predictions.join(valid_target)

In [41]:
# Создаем маску, где true - если expected присутствует среди id_predicted для каждого id_query
mask = merged_valid.groupby(merged_valid.index).apply(lambda x: x['expected'].iloc[0] in x['id_predicted'].values)

# Вычисляем accuracy как процент true значений в маске
accuracy_at_50 = (mask.sum() / len(mask)) * 100
print(f"Точность 50 соседей на валидационной выборке: {accuracy_at_50}%")

Точность 50 соседей на валидационной выборке: 76.696%


In [42]:
# Преобразование матрицы valid в DataFrame
valid_df = pd.DataFrame(valid)
valid_df.index = [valid_index[i] for i in range(valid.shape[0])]

merged_valid_df = merged_valid.join(valid_df)
merged_valid_df = pd.merge(merged_valid_df, base_transform_df, left_on='id_predicted', right_index=True, how='left')

In [43]:
merged_valid_df['expected'] = (merged_valid_df['expected'] == merged_valid_df['id_predicted']).astype(int)

In [44]:
target_merged_valid = merged_valid_df['expected']
features_merged_valid = merged_valid_df.drop(['id_predicted', 'expected'] , axis=1)

In [45]:
del valid, train, base, idx_df, train_new, merged_valid, D, I, mask

Вывод: 

Создали валидационную выборку на основе поиска 50 ближайших соседей.

### Модель Логистической регресии

Обучим модель логистичиской регрессии. Таргет признак - совпадение предсказанного id товара с id товара размеченного. Для каждого товара мы нашли 50 кандидатов и среди них у 76% есть размеченный товар. \
Модель логистической регрессии будет предсказывать случаи, когда id векторов совпадают то класс 1.

In [46]:
# Модель логистической регрессии
model = LogisticRegression(solver='lbfgs', class_weight='balanced', random_state=42)
model.fit(features_train.values, target_train)

LogisticRegression(class_weight='balanced', random_state=42)

In [47]:
valid_predictions = model.predict(features_merged_valid.values)
valid_probabilities = model.predict_proba(features_merged_valid.values)[:, 1]
# Вычисление точности на валидационном наборе
valid_accuracy = accuracy_score(target_merged_valid, valid_predictions)

In [48]:
print('Точность на Валидационной выборке', valid_accuracy)

Точность на Валидационной выборке 0.6010702


In [49]:
valid_proba_series = pd.Series(valid_probabilities)
valid_candidates = []
for i in range(0, len(valid_proba_series), N_NEIGHBORS):
    query = valid_proba_series[i : i+N_NEIGHBORS]
    index = query.sort_values(ascending=False)[0:5].index
    valid_candidates.append(index)

In [50]:
valid_candidates_df = pd.DataFrame(data=valid_candidates)
# Преобразование pd.Series в pd.DataFrame
target_merged_valid_df = target_merged_valid.to_frame()

target_merged_valid_df['prob'] = valid_proba_series.values

target_merged_valid_df['prediction'] = valid_predictions
# Преобразование индекса в столбец
target_merged_valid_df_reset = target_merged_valid_df.reset_index().rename(columns={'index': 'id_query'})

# Группировка по 'id_query' и отбор топ-5 записей с наибольшими вероятностями
top_5_per_group = target_merged_valid_df_reset.groupby('id_query').apply(
    lambda x: x.nlargest(5, 'prob')
)

top_5_per_group = top_5_per_group.reset_index(drop=True)

In [51]:
# Для каждой группы проверяем, есть ли среди топ-5 записей такие, где expected и prediction равны 1
accuracy_per_group = top_5_per_group.groupby('id_query').apply(
    lambda x: ((x['expected'] == 1) & (x['prediction'] == 1)).any()
).mean()

In [52]:
print(f'Accuracy среди топ-5 кандидатов: {accuracy_per_group:.2%}')

Accuracy среди топ-5 кандидатов: 12.81%


Запомним это значение, в будущих обновлнниях попробуем еще модель для сравнения.

## Вывод:


В данном проекте мы проделали следующую работу
* Загрузка и предобработка данных: На этом этапе мы загрузили данные, очистили их от возможных аномалий и пропусков, подготовив таким образом к дальнейшему анализу.

* Масштабирование данных: Следующим шагом было масштабирование признаков для улучшения работы алгоритмов машинного обучения и упрощения процесса обучения модели.

* Применение Faiss для векторного поиска: Использование Faiss позволило нам эффективно разделить векторное пространство на 50 кластеров и найти 50 ближайших соседей для каждого товара из запроса. Это дало возможность ускорить процесс поиска и сделать его более масштабируемым.

* Оценка точности подбора соседей: Используя размеченные данные, мы выяснили, что среди 50 подобранных соседей для каждого товара из запроса у 76% товаров было найдено совпадение по размеченному признаку. Это свидетельствует о высокой точности алгоритма в поиске релевантных товаров.

* Применение модели логистической регрессии: Дальнейшее использование логистической регрессии для выбора 5 наиболее подходящих кандидатов из 50 предварительно отобранных показало долю правильных подборов в 13%. Этот результат указывает на потенциал для улучшения и оптимизации процесса метчинга.

### Возможные направления для улучшений:
Использование более сложных моделей машинного обучения: Вместо логистической регрессии можно применить более сложные и мощные алгоритмы, такие как случайный лес, градиентный бустинг или нейронные сети, которые могут улучшить точность предсказаний.

Расширение набора признаков: Добавление новых признаков, таких как текстовые описания товаров, категории, ценовые диапазоны, может помочь моделям лучше различать товары и повысить точность метчинга.

Оптимизация гиперпараметров: Тонкая настройка гиперпараметров существующих моделей с помощью методов, таких как GridSearchCV или RandomizedSearchCV, может привести к значительному улучшению результатов.

Использование методов глубокого обучения: Применение свёрточных нейронных сетей для анализа изображений товаров или рекуррентных нейронных сетей для обработки их описаний может улучшить способность моделей выделять ключевые характеристики товаров.

In [65]:
del valid_candidates, query, valid_predictions, valid_probabilities, valid_candidates_df, valid_proba_series